In [ ]:
!pip install -q tfds-nightly

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow_datasets as tfds

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/ModelCNN_WEBIA.keras')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
  builder = tfds.builder("eurosat")

In [ ]:
ds, info = tfds.load("eurosat",
                    split="train",
                    with_info=True,
                    builder_kwargs={"config": "rgb"})  # On utilise seulement les données RGB (Red, Green, Blue) de Sentinel 2
                                                       # Ce sont les bandes 2,3 et 4

In [ ]:
def preprocess(sample):
    image = sample['image']
    label = sample['label']


    # Ici, nous normalisons les données pour qu'elles soient plus facilement traitable
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [ ]:
batch_size = 32
train_size = int(0.8 * info.splits["train"].num_examples)

train_ds = (ds.take(train_size)
            .map(preprocess)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE))

val_ds = (ds.skip(train_size)
          .map(preprocess)
          .batch(batch_size)
          .prefetch(tf.data.AUTOTUNE))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 64, 3)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"), # Couche de convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(), # Transforme les résultat en un seul vecteur
    tf.keras.layers.Dense(256, activation='relu'), # Couche dense (chaque perceprton est relié à un autre de la couche Flatten), l'activation ReLU veut dire qu'on garde que les valeurs positives
    tf.keras.layers.Dropout(0.5), # Met 50% des poids à 0 pour evitter l'overfitting
    tf.keras.layers.Dense(10, activation='softmax') # Couche dense qui donne une proba d'appartenance sur les 10 catégorie (la somme des probas est égale à 1)(c'est à ca que sert l'activation softmax)
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=20)
model.save('/content/drive/MyDrive/ModelCNN_WEBIA.keras')